# IHLT Lab 4: Part of Speech

**Authors:** *Zachary Parent ([zachary.parent](mailto:zachary.parent@estudiantat.upc.edu)), Carlos Jiménez ([carlos.humberto.jimenez](mailto:carlos.humberto.jimenez@estudiantat.upc.edu))*

### 2024-10-10

**Instructions:**

1. Consider Treebank corpus.

    - Train HMM, TnT, perceptron and CRF models using the first 500, 1000, 1500, 2000, 2500 and 3000 sentences.

    - Evaluate the resulting 24 models using sentences from 3001.

2. Provide a figure with four learning curves, each per model type (X=training set size; Y=accuracy).

    - Which model would you select? Justify the answer.


## Notes


In [1]:
import nltk
nltk.download('treebank')

[nltk_data] Downloading package treebank to
[nltk_data]     /home/carlos.jimenez/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [2]:
len(nltk.corpus.treebank.tagged_sents())

3914

In [3]:
nltk.corpus.treebank.tagged_sents()[1]

[('Mr.', 'NNP'),
 ('Vinken', 'NNP'),
 ('is', 'VBZ'),
 ('chairman', 'NN'),
 ('of', 'IN'),
 ('Elsevier', 'NNP'),
 ('N.V.', 'NNP'),
 (',', ','),
 ('the', 'DT'),
 ('Dutch', 'NNP'),
 ('publishing', 'VBG'),
 ('group', 'NN'),
 ('.', '.')]

In [4]:
## Learning the model

def hidden_markov(train, test):
    def LID(fd, bins):
        return nltk.probability.LidstoneProbDist(fd, 0.1, bins)
    
    trainer = nltk.tag.hmm.HiddenMarkovModelTrainer()
    HMM = trainer.train_supervised(train, estimator=LID)
    acc = HMM.accuracy(test)
    print(f'HMM acc: {acc}')
    return acc

# set(test)
# len(set(test).difference(train))

In [5]:
# HMM = nltk.HiddenMarkovModelTagger.train(train)
# HMM.accuracy(test)

## TnT

In [6]:

def TnT(train, test):
    TnT = nltk.tag.tnt.TnT()
    TnT.train(train)
    acc = TnT.accuracy(test)
    print(f'TnT acc: {acc}')
    return acc
#     TnT.tag(['the', 'men', 'attended', 'to', 'the', 'meetings'])

# Perceptron

In [7]:
def perceptron(train, test):
    PER = nltk.tag.perceptron.PerceptronTagger(load=False)
    PER.train(train)
    acc = PER.accuracy(test)
    print(f'PER acc: {acc}')
    return acc
# PER.tag(['the', 'men', 'attended', 'to', 'the', 'meetings']) 

# CRF

In [8]:
# !pip install python-crfsuite

In [9]:
def CRF(train, test):
    CRF = nltk.tag.CRFTagger()
    CRF.train(train,'crf_tagger_model')
    acc = CRF.accuracy(test)
    
    print(f'CRF acc: {acc}')
    
    return acc
    # CRF.tag(['the', 'men', 'attended', 'to', 'the', 'meetings'])

## Train all models with different sentences number

In [10]:
# Sentences number list
sentences_n = [500, 1000, 1500, 2000, 2500, 3000]

In [ ]:
test = nltk.corpus.treebank.tagged_sents()[3000:]

for n in sentences_n:
    print(f'Training with {n} sentences...')
    train = nltk.corpus.treebank.tagged_sents()[:n]
    
    hmm_acc = hidden_markov(train, test)
    tnt_acc = TnT(train, test)
    per_acc = perceptron(train, test)
    crf_acc = CRF(train, test)
    
    print()
    

Training with 500 sentences...
HMM acc: 0.793654219728038
TnT acc: 0.7472911720267645
PER acc: 0.9126268076840061
CRF acc: 0.9095618389812217

Training with 1000 sentences...
HMM acc: 0.8347938700625944
TnT acc: 0.7962875026980358
PER acc: 0.9325275199654651
CRF acc: 0.9249298510684222

Training with 1500 sentences...
HMM acc: 0.8594431254047054


## Do your thing Zach with the plots ;)

## TODO:
"""

Provide a figure with four learning curves, each per model type (X=training set size; Y=accuracy).

Which model would you select? Justify the answer.

"""